In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns
import scipy.special as sp
import sklearn.datasets as datasets
import sklearn.linear_model as lm
import sklearn.ensemble as en
import sklearn.tree as tree
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
import econml
import importlib
import pyreadr

#### data generation packages
import strawman_edge
import strawman_center
import all_linear

#### main packages
import characterize
import estimate_atte as est
import learn_w as learn
import black

importlib.reload(est)
importlib.reload(learn)

import warnings

warnings.filterwarnings("ignore")

%load_ext jupyter_black
sns.set(font_scale=1.25, style="whitegrid")
np.random.seed(0)

In [ ]:
outcome_cols = ["opioiduse12", "opioiduse24"]
treatment_col = "medicine_assigned"
discrete_cov = ["xrace", "mar", "sex"]

baseline_harmonized = pd.read_csv(
    "/Users/harshparikh/Library/CloudStorage/OneDrive-JohnsHopkins/MOUD_data/updated_data/ctn0094/drv/clean_patients_with_relapse_wide.csv",
    index_col=0,
)

# baseline_harmonized = baseline_harmonized.rename(
#     columns={col: col.split(".")[1] for col in baseline_harmonized.columns}
# ).set_index("who")

stacked_list = []
for i in range(1, 6):
    stacked_list.append(
        pd.read_csv(
            "/Users/harshparikh/Library/CloudStorage/OneDrive-JohnsHopkins/MOUD_data/stacked_list_%d.csv"
            % (i),
            index_col=0,
        )
    )
df = stacked_list[0]

df_tedsa = df.loc[df["trialdata"] == 0]

## TEDS-A vs CTN 27

In [ ]:
ct94 = baseline_harmonized.loc[(baseline_harmonized["project"] == 27)]
outcome94 = ct94[outcome_cols]

common_cols = set.intersection(set(df_tedsa.columns), set(ct94.columns))

ct94_cc = ct94[common_cols].drop(columns=["edu", "mar"])
ct94_cc["sex"] = (ct94["sex"] == "male").astype(int)  # male = 1 and female = 0
# ct94_cc["mar"].loc[(1 - ct94_cc["mar"].isna()) == 1] = (ct94_cc["mar"] == 1.0).astype(
#     int
# )
# ct94_cc["edu"].loc[(1 - ct94_cc["edu"].isna()) == 1] = (ct94_cc["edu"] >= 2.0).astype(
#     int
# )

imputer = KNNImputer(n_neighbors=4, weights="distance", add_indicator=False)
ct94_cc_imputed = imputer.fit_transform(ct94_cc)

ct94_cc = pd.DataFrame(ct94_cc_imputed, index=ct94_cc.index, columns=ct94_cc.columns)

ct94_cc["med_met"] = (ct94[treatment_col] == "met").astype(
    int
)  # methadone = 1 and bupenorphine = 0
ct94_cc = ct94_cc.dropna()

ct94_cc["S"] = 1
ct94_cc = ct94_cc.round(0).astype(int)

ct94_cc = ct94_cc.join(outcome94, how="inner")
print(ct94_cc.shape)
ct94_cc.groupby(by="med_met").mean()[outcome_cols]

In [ ]:
df_tedsa_cc = df_tedsa[common_cols].drop(columns=["edu", "mar"])
df_tedsa_cc["S"] = 0
# invert age categories
df_tedsa_cc["age"].replace(
    {
        1: 13,
        2: 16,
        3: 18,
        4: 22,
        5: 27,
        6: 32,
        7: 37,
        8: 42,
        9: 47,
        10: 52,
        11: 60,
        12: 68,
    },
    inplace=True,
)

# df_tedsa_cc["mar"].loc[(1 - df_tedsa_cc["mar"].isna()) == 1] = (
#     df_tedsa_cc["mar"] == 1.0
# ).astype(int)
# df_tedsa_cc["edu"].loc[(1 - df_tedsa_cc["edu"].isna()) == 1] = (
#     df_tedsa_cc["edu"] >= 2.0
# ).astype(int)

In [ ]:
df_primary = pd.concat([df_tedsa_cc.sample(frac=1, replace=False), ct94_cc])

df_ = df_primary.drop(columns=[outcome_cols[0]]).fillna(0)

In [ ]:
outcome = outcome_cols[1]
treatment = "med_met"
sample = "S"
data = df_
S = df_[sample]  # indicator for the sample
Y = df_[outcome]  # outcome variable
T = df_[treatment]  # indicator for the treatment

data_dummy = pd.get_dummies(data, columns=["xrace"])
data_dummy.rename(
    columns={
        "sex": "Male",
        "age": "Age",
        "ivdrug": "IV Drug Use",
        "bamphetamine30_base": "Hx Amphetamine",
        "bbenzo30_base": "Hx Benzo",
        "bcannabis30_base": "Hx Cannabis",
        "xrace_1": "White",
        "xrace_2": "Black",
        "xrace_3": "Hispanic",
        "xrace_4": "Other Race",
        # "mar": "Married",
        # "edu": "Education(12y)",
    },
    inplace=True,
)

X = data_dummy.drop(columns=[outcome, treatment, sample])  # pre-treatment covariates

In [ ]:
print(data_dummy.groupby(by=[sample]).quantile(0.75).T.round(2).to_latex())

#### CTN Analysis

In [ ]:
importlib.reload(learn)
df_v, pi, pi_m, e_m, data2 = learn.estimate_dml(
    data_dummy, outcome, treatment, sample, crossfit=5
)

In [ ]:
pd.concat(
    [
        df_v[["a", "te"]].mean() * 100,
        np.sqrt(df_v[["a_sq", "te_sq"]].mean() / df_v.shape[0]) * 100,
    ],
)

In [ ]:
np.random.seed(42)
pi_m = en.AdaBoostRegressor().fit(X.drop(columns=["Age"]), S)
exp_te_m = en.AdaBoostRegressor().fit(data2[X.drop(columns=["Age"]).columns], df_v["a"])

In [ ]:
pd.concat(
    [
        pd.DataFrame(
            pi_m.feature_importances_,
            index=X.drop(columns=["Age"]).columns,
            columns=["Sample Selector"],
        ),
        pd.DataFrame(
            exp_te_m.feature_importances_,
            index=X.drop(columns=["Age"]).columns,
            columns=["Effect Modifier"],
        ),
    ],
    axis=1,
).round(2)

In [ ]:
X.loc[S == 1].corr()

#### Transporting Effects

In [ ]:
importlib.reload(learn)
# D, w_tree, testing_data = learn.tree_opt(data, outcome, treatment, sample, leaf_proba=1)
np.random.seed(42)
D_rash, D_forest, w_forest, rashomon_set, f, testing_data = learn.forest_opt(
    data=data_dummy,
    outcome=outcome,
    treatment=treatment,
    sample=sample,
    leaf_proba=0.5,
    num_trees=5000,
    vote_threshold=5 / 10,
    # top_k_trees=True,
    # k=5,
)

baseline_loss = np.sqrt(np.sum(D_forest["vsq"]) / ((D_forest.shape[0] ** 2)))
# print("Baseline Objective:%.4f" % (baseline_loss))

In [ ]:
local_obj = pd.DataFrame(
    np.array([w_forest[i]["local objective"] for i in range(len(w_forest))]),
    columns=["Objective"],
).sort_values(by="Objective")

top_k = 3000
plt.plot((local_obj.iloc[:top_k] / baseline_loss)["Objective"].values)

In [ ]:
w_rash = [
    "w_tree_%d" % (i)
    for i in range(len(w_forest))
    if i in list(local_obj.iloc[:top_k].index)
]
avg_votes = (D_forest[w_rash].mean(axis=1) >= 0.5).astype(int)

np.random.seed(42)
num_trees = 1
explainer = tree.DecisionTreeClassifier(max_leaf_nodes=16).fit(
    X.drop(columns=["Age"]).loc[avg_votes.index], avg_votes
)

In [ ]:
fig, ax = plt.subplots(nrows=num_trees, figsize=(10, 5), dpi=600)
for i in range(num_trees):
    if num_trees == 1:
        tree.plot_tree(
            explainer,  # .estimators_[i, 0],
            feature_names=X.drop(columns=["Age"]).columns,
            ax=ax,
            filled=True,
            fontsize=10,
            proportion=True,
            impurity=False,
        )
    else:
        tree.plot_tree(
            explainer.estimators_[i, 0],
            feature_names=X.drop(columns=["Age"]).columns,
            ax=ax[i],
            filled=True,
            fontsize=10,
            proportion=True,
        )
plt.savefig("tedsa_ctn27.pdf", dpi=600)

In [ ]:
w = explainer.predict(data_dummy[X.drop(columns=["Age"]).columns])
data_dummy["w"] = w

pd.concat(
    [
        data_dummy.groupby(by="S").mean().T.round(2),
        data_dummy.loc[data_dummy["w"] == 1].groupby(by="S").mean().T.round(2),
    ],
    axis=1,
) * 100

In [ ]:
print(
    r"Pre Pruning, SATE: %.2f ± %.2f"
    % (
        100
        * (
            df_.loc[(df_[sample] == 1) * (df_[treatment] == 1), outcome].mean()
            - df_.loc[(df_[sample] == 1) * (df_[treatment] == 0), outcome].mean()
        ),
        100
        * 1.96
        * (
            df_.loc[(df_[sample] == 1) * (df_[treatment] == 1), outcome].sem()
            + df_.loc[(df_[sample] == 1) * (df_[treatment] == 0), outcome].sem()
        ),
    )
)

print(
    r"Pre Pruning, ATTE: %.2f ± %.2f"
    % (
        100 * D_rash["v"].mean(),
        100 * 1.96 * np.sqrt(D_rash["vsq"].sum() / (D_rash["w_opt"].count()) ** 2),
    )
)

D_rash["w_opt"] = explainer.predict(X.drop(columns=["Age"]).loc[avg_votes.index])
print(
    r"Post Pruning, ATTE: %.2f ± %.2f"
    % (
        100 * D_rash["v"].loc[D_rash["w_opt"] == 1].mean(),
        100
        * 1.96
        * np.sqrt(
            D_rash["vsq"].loc[D_rash["w_opt"] == 1].sum()
            / ((D_rash["w_opt"].sum()) ** 2)
        ),
    )
)

In [ ]:
sns.kdeplot(D_rash.loc[D_rash["w_opt"] == 1]["vsq"], fill=True)
sns.kdeplot(D_rash.loc[D_rash["w_opt"] == 0]["vsq"], fill=True)
plt.axvline(D_rash.loc[D_rash["w_opt"] == 1]["vsq"].mean(), c="C0")
plt.axvline(D_rash.loc[D_rash["w_opt"] == 0]["vsq"].mean(), c="C1")
plt.xlim(0, 95)
print(
    (
        np.sqrt(
            D_rash.loc[data_dummy["w"] == 1]["vsq"].mean()
            / D_rash.loc[data_dummy["w"] == 1]["vsq"].count()
        ),
        np.sqrt(D_rash["vsq"].mean() / D_rash["vsq"].count()),
    )
)

In [ ]:
sns.kdeplot(D_rash.loc[D_rash["w_opt"] == 1]["v"], fill=True)
sns.kdeplot(D_rash.loc[D_rash["w_opt"] == 0]["v"], fill=True)
print(
    (
        D_rash.loc[D_rash["w_opt"] == 1]["v"].mean(),
        D_rash.loc[D_rash["w_opt"] == 0]["v"].mean(),
    )
)